# Homework 6: Deep Convolutional Neural Networks

Ở bài tập này ta sẽ thực hành lập trình một vài mạng nơ-ron tích chập (CNN) giải quyết 2 bài toán: Phân loại phương tiện giao thông, Phân loại biển báo giao thông.

Yêu cầu thư viện: OpenCV 3.3, Keras 2.2.2, TensorFlow 1.11.0 

In [5]:
import keras
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2

In [6]:
from keras.layers import Input
from keras.models import Model

## Yêu cầu: Lập trình mạng VGG (2014) với Keras

Hãy sử dụng những hàm bạn vừa được học ở trên để xây dựng một kiến trúc mạng nơ-ron tích chập nổi tiếng khác là VGG (2014). Tuy nhiên vì mạng VGG có kích thước khá lớn nên để phù hợp với bài toán khảo sát ta sẽ thử lập trình một phiên bản khác của VGG có tên BKNet (D.Sang et al, 2017). Hãy quan sát cấu trúc của BKNet dưới đây:

<img src="BKNet1.png" width=300>

Hãy lập trình hàm BKNet() nhận đầu vào là kích thước ảnh, số lượng lớp và weight_decay tương tự như các hàm LeNet ở trên để xây dựng mô hình mạng CNN BKNet. Ta tạm thời bỏ qua lớp BatchNorm. Bạn có thể sử dụng Sequential model hoặc FunctionAPI.

In [7]:
def BKNet(img_shape=(96, 96, 3), num_classes=17, weight_decay=1e-4):
    #### YOUR CODE HERE ####
    
    # Input layer
    input = Input(img_shape)
    
    # Layer 1
    x = Conv2D(filters=32, kernel_size=3, strides=1, padding='same', kernel_regularizer=l2(weight_decay),
                     activation='relu')(input)
    x = Conv2D(filters=32, kernel_size=3, strides=1, padding='same', kernel_regularizer=l2(weight_decay),
                     activation='relu')(x)
    x = MaxPooling2D(pool_size=2, strides=2, padding='same')(x)

    # Layer 2
    x = Conv2D(filters=64, kernel_size=3, strides=1, padding='same', kernel_regularizer=l2(weight_decay),
               activation='relu')(x)
    x = Conv2D(filters=64, kernel_size=3, strides=1, padding='same', kernel_regularizer=l2(weight_decay),
               activation='relu')(x)
    x = MaxPooling2D(pool_size=2, strides=2, padding='same')(x)
    
    # Layer 3
    x = Conv2D(filters=128, kernel_size=3, strides=1, padding='same', kernel_regularizer=l2(weight_decay),
               activation='relu')(x)
    x = Conv2D(filters=128, kernel_size=3, strides=1, padding='same', kernel_regularizer=l2(weight_decay),
               activation='relu')(x)
    x = MaxPooling2D(pool_size=2, strides=2, padding='same')(x)
    
    # Layer 4
    x = Conv2D(filters=256, kernel_size=3, strides=1, padding='same', kernel_regularizer=l2(weight_decay),
               activation='relu')(x)
    x = Conv2D(filters=256, kernel_size=3, strides=1, padding='same', kernel_regularizer=l2(weight_decay),
               activation='relu')(x)
    x = Conv2D(filters=256, kernel_size=3, strides=1, padding='same', kernel_regularizer=l2(weight_decay),
               activation='relu')(x)
    x = MaxPooling2D(pool_size=2, strides=2, padding='same')(x)
    
    # Layer 5
    x = Flatten()(x)
    x = Dense(units=256, activation='relu', kernel_regularizer=l2(weight_decay))(x)
    x = Dense(units=256, activation='relu', kernel_regularizer=l2(weight_decay))(x)
    
    # Layer 5
    output = Dense(units=num_classes, activation='softmax')(x)

    BKNet = Model(inputs=input, outputs=output)

    return BKNet

    #### END YOUR CODE #####

In [8]:
my_BKNet = BKNet(img_shape=(96, 96, 3), num_classes=17, weight_decay=1e-4)
my_BKNet.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 96, 96, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 96, 96, 32)        896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 96, 32)        9248      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 48, 48, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 24, 24, 64)       

# Huấn luyện mạng nơ-ron tích chập cho bài toán Phân loại phương tiện 

Ở bài tập này ta sẽ sử dụng các mô hình CNN đã được định nghĩa ở trên cho bài toán Phân loại phương tiện. Bộ dữ liệu của bài toán được đặt ở thư mục './Dataset/vehicle'. Trong thư mục này ta có 2 thư mục con: Training, Testing tương ứng với dữ liệu để huấn luyện mô hình và dữ liệu để kiểm tra độ chính xác của mô hình. Cấu trúc các 2 thư mục này tương tự nhau: Trong mỗi thư mục có các thư mục con ứng với các lớp của bài toán, trong mỗi thư mục ứng với các lớp lại chứa các ảnh ứng với lớp đó. 

Hàm prepair_data() dưới đây nhận các đầu vào:
- path: đường dẫn đến bộ dữ liệu
- img_shape: kích thước ảnh mong muốn

và trả lại 4 mảng:
- x_train, x_test: Kích thước (N, img_shape[0], img_shape[1], 3) chứa dữ liệu ảnh.
- y_train, y_test: Kích thước (N, num_classes) chứa các one-hot vector biểu diễn nhãn của ảnh. num_classes được tính dựa vào số thư mục con trong thư mục Training/Testing

In [9]:
!wget 'https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/17018/799209/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1607176215&Signature=MdKdgod%2BrHFzW16Nov93czl2e1SN6gIFKyAYvI9Fah6JHqo%2FjA%2BB8G8oTqHvgnZ%2BJmtF3xRfTSBm8cUaVG7idKT%2FQRK55xNUeoy1573nyclrOrR7IcqfAAemmXqNlpS5kq4%2BPM%2Fj1XcCiRm6Bl5%2FSNWR0OdqhrJpydhl%2BJ5lmf%2FhQh08lyU%2FbAiloVFfJZfVxmr0gKSncXxQkVG%2FZFUR16qjGQVBpKQCyvxBdUMxRahl902ZkAcjy4Q6HfdkbVgjHO%2BGdWLUmEMrBsUwWWFi8sIPXHx9OUrJxcV7gL2KWbtqoeNtlmNRvWx3HfGrCnYtRQyD9Y4N3fDUn5OTRcldmg%3D%3D&response-content-disposition=attachment%3B+filename%3Dvehicle.zip'


The name is too long, 523 chars total.
Trying to shorten...
New name is archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1607176215&Signature=MdKdgod+rHFzW16Nov93czl2e1SN6gIFKyAYvI9Fah6JHqo%2FjA+B8G8oTqHvgnZ+JmtF3xRfTSBm8cUaVG7idKT%2FQRK55xNUeoy1573nyclrOrR7IcqfAAemmXqNlpS5k.
--2020-12-02 15:36:47--  https://storage.googleapis.com/kaggle-competitions-data/kaggle-v2/17018/799209/bundle/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1607176215&Signature=MdKdgod%2BrHFzW16Nov93czl2e1SN6gIFKyAYvI9Fah6JHqo%2FjA%2BB8G8oTqHvgnZ%2BJmtF3xRfTSBm8cUaVG7idKT%2FQRK55xNUeoy1573nyclrOrR7IcqfAAemmXqNlpS5kq4%2BPM%2Fj1XcCiRm6Bl5%2FSNWR0OdqhrJpydhl%2BJ5lmf%2FhQh08lyU%2FbAiloVFfJZfVxmr0gKSncXxQkVG%2FZFUR16qjGQVBpKQCyvxBdUMxRahl902ZkAcjy4Q6HfdkbVgjHO%2BGdWLUmEMrBsUwWWFi8sIPXHx9OUrJxcV7gL2KWbtqoeNtlmNRvWx3HfGrCnYtRQyD9Y4N3fDUn5OTRcldmg%3D%3D&response-content-disposition=attachment%3B+filename%3Dvehicle.zip
Resolving storage.googleapis.com (s

In [10]:
!unzip '/content/archive.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1607176215&Signature=MdKdgod+rHFzW16Nov93czl2e1SN6gIFKyAYvI9Fah6JHqo%2FjA+B8G8oTqHvgnZ+JmtF3xRfTSBm8cUaVG7idKT%2FQRK55xNUeoy1573nyclrOrR7IcqfAAemmXqNlpS5k'

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: train/train/Motorcycle/003967_09.jpg  
  inflating: train/train/Motorcycle/003972_10.jpg  
  inflating: train/train/Motorcycle/003975_17.jpg  
  inflating: train/train/Motorcycle/003977_18.jpg  
  inflating: train/train/Motorcycle/003982_17.jpg  
  inflating: train/train/Motorcycle/003983_09.jpg  
  inflating: train/train/Motorcycle/003989_10.jpg  
  inflating: train/train/Motorcycle/003990_01.jpg  
  inflating: train/train/Motorcycle/003992_05.jpg  
  inflating: train/train/Motorcycle/003992_17.jpg  
  inflating: train/train/Motorcycle/003993_09.jpg  
  inflating: train/train/Motorcycle/003993_18.jpg  
  inflating: train/train/Motorcycle/003997_00.jpg  
  inflating: train/train/Motorcycle/003998_02.jpg  
  inflating: train/train/Motorcycle/003998_18.jpg  
  inflating: train/train/Motorcycle/003999_01.jpg  
  inflating: train/train/Motorcycle/004004_02.jpg  
  inflating: train/train/Motorcycle/004004_05.jpg  
  infla

In [11]:
import os
labels = os.listdir('/content/train/train/')
labels

['Motorcycle',
 'Boat',
 'Truck',
 'Helicopter',
 'Ambulance',
 'Segway',
 'Car',
 'Tank',
 'Bus',
 'Limousine',
 'Barge',
 'Snowmobile',
 'Bicycle',
 'Taxi',
 'Cart',
 'Van',
 'Caterpillar']

In [12]:
os.mkdir('test_new') 

In [13]:
for l in labels:
  os.mkdir('test_new/'+l)

In [14]:
def split_train_test(label_name, proportion_of_train):
  train_path = '/content/train/train/'+label_name+'/'
  test_path = '/content/test_new/'+label_name+'/'
  all_file_name = os.listdir(train_path)
  num_of_train = int(len(all_file_name)*proportion_of_train)
  for i in range(num_of_train, len(all_file_name)):
    os.rename(train_path+all_file_name[i], test_path+all_file_name[i])

In [15]:
for i in labels:
  split_train_test(i, 0.8)

In [16]:
import numpy as np
import os
import cv2

def prepair_data(path='', img_shape=(32, 32)):
    TRAINING_DATA_PATH = os.path.join(path, 'train/train')
    TESTING_DATA_PATH = os.path.join(path, 'test_new')

    x_train = []
    y_train = []
    x_test = []
    y_test = []

    label_id = 0
    
    num_classes = len(os.listdir(TRAINING_DATA_PATH))
    for label in os.listdir(TRAINING_DATA_PATH):

        # Read training data
        for img_file in os.listdir(os.path.join(TRAINING_DATA_PATH, label)):
            img = cv2.imread(os.path.join(TRAINING_DATA_PATH, label, img_file))
            img = cv2.resize(img, img_shape)
            x_train.append(img)

            y = np.zeros(num_classes)
            y[label_id] = 1
            y_train.append(y)

        # Read testing data
        for img_file in os.listdir(os.path.join(TESTING_DATA_PATH, label)):
            img = cv2.imread(os.path.join(TESTING_DATA_PATH, label, img_file))
            img = cv2.resize(img, img_shape)
            x_test.append(img)

            y = np.zeros(num_classes)
            y[label_id] = 1
            y_test.append(y)

        label_id += 1

    return np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)

In [37]:
TRAINING_DATA_PATH = os.path.join('', 'train/train')
TESTING_DATA_PATH = os.path.join('', 'test_new')
TRAINING_DATA_PATH

'train/train'

In [17]:
x_train, y_train, x_test, y_test = prepair_data(path='', img_shape=(32, 32))
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(22427, 32, 32, 3)
(22427, 17)
(5618, 32, 32, 3)
(5618, 17)


Bây giờ ta sẽ thử huấn luyện mạng LeNet đã định nghĩa ở trên:

In [18]:
BKNet = BKNet(img_shape=(32, 32, 3), num_classes=17)
BKNet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

BKNet.fit(x_train, y_train, epochs=20, batch_size=16)

Epoch 1/20
1402/1402 [==============================] - 9s 6ms/step - loss: 1.9267 - accuracy: 0.4198
Epoch 2/20
1402/1402 [==============================] - 8s 6ms/step - loss: 1.5775 - accuracy: 0.5319
Epoch 3/20
1402/1402 [==============================] - 8s 6ms/step - loss: 1.4386 - accuracy: 0.5779
Epoch 4/20
1402/1402 [==============================] - 8s 6ms/step - loss: 1.3562 - accuracy: 0.6102
Epoch 5/20
1402/1402 [==============================] - 9s 6ms/step - loss: 1.3002 - accuracy: 0.6238
Epoch 6/20
1402/1402 [==============================] - 9s 6ms/step - loss: 1.2599 - accuracy: 0.6409
Epoch 7/20
1402/1402 [==============================] - 9s 6ms/step - loss: 1.2216 - accuracy: 0.6589
Epoch 8/20
1402/1402 [==============================] - 9s 6ms/step - loss: 1.1877 - accuracy: 0.6706
Epoch 9/20
1402/1402 [==============================] - 9s 6ms/step - loss: 1.1652 - accuracy: 0.6797
Epoch 10/20
1402/1402 [==============================] - 9s 6ms/step - loss: 1.145

Test model đã huấn luyện với dữ liệu test:

In [19]:
BKNet.evaluate(x_test, y_test)

176/176 [==============================] - 1s 4ms/step - loss: 1.4737 - accuracy: 0.6613


[1.4736578464508057, 0.6612673401832581]

Cuối cùng ta lưu lại mô hình đã huấn luyện để có thể sử dụng sau này:

In [ ]:
from keras.models import model_from_json

# Save architecture and weight
with open('BKNet.json', 'w') as f:
    f.write(BKNet.to_json())
    
BKNet.save_weights("BKNet_weight.h5")

# Load trained model
json_file = open('BKNet.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)

loaded_model.load_weights("BKNet_weight.h5")
loaded_model.summary()

# Re-compile and evaluate model
loaded_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
loaded_model.evaluate(x_test, y_test)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 28, 28, 6)         456       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_32 (Conv2D)           (None, 10, 10, 16)        2416      
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 5, 5, 16)          0         
_________________________________________________________________
flatten_9 (Flatten)          (None, 400)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 120)               48120     
__________

[1.260407393584486, 0.7704918045815223]

## Huấn luyện mạng nơ-ron tích chập cho bài toán Phân loại biển báo giao thông

Tương tự như bài tập phân loại phương tiện giao thông ở trên, hãy thực hành với bài toán Phân loại biển báo giao thông sử dụng LeNet. Dữ liệu của bài toán được lưu ở thư mục './Dataset/sign' có cấu trúc tương tự như dữ liệu bài toán phân loại phương tiện giao thông. 

In [ ]:
# Prepair data
x_train, y_train, x_test, y_test = None, None, None, None

#### YOUR CODE HERE ####
x_train, y_train, x_test, y_test = prepair_data(path='./Dataset/sign', img_shape=(32, 32))
#### END YOUR CODE #####

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(4575, 32, 32, 3)
(4575, 62)
(2520, 32, 32, 3)
(2520, 62)


In [ ]:
# Train, test and save your model

BKNet = BKNet(img_shape=(32, 32, 3), num_classes=62)
BKNet.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

BKNet.fit(x_train, y_train, epochs=40, batch_size=16)

Epoch 1/40
4575/4575 [==============================] - 3s 640us/step - loss: 4.1481 - acc: 0.4916
Epoch 2/40
4575/4575 [==============================] - 2s 521us/step - loss: 0.7686 - acc: 0.8645
Epoch 3/40
4575/4575 [==============================] - 2s 504us/step - loss: 0.4186 - acc: 0.9412
Epoch 4/40
4575/4575 [==============================] - 2s 516us/step - loss: 0.3540 - acc: 0.9591
Epoch 5/40
4575/4575 [==============================] - 2s 513us/step - loss: 0.3122 - acc: 0.9694
Epoch 6/40
4575/4575 [==============================] - 3s 611us/step - loss: 0.3187 - acc: 0.9718
Epoch 7/40
4575/4575 [==============================] - 3s 619us/step - loss: 0.3076 - acc: 0.9711
Epoch 8/40
4575/4575 [==============================] - 2s 451us/step - loss: 0.2838 - acc: 0.9810
Epoch 9/40
4575/4575 [==============================] - 3s 671us/step - loss: 0.2941 - acc: 0.9742
Epoch 10/40
4575/4575 [==============================] - 3s 759us/step - loss: 0.3294 - acc: 0.9696
Epoch 11/